In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define default paths (relative paths inside the project folder)
csv_file = "data/Labels.csv"
noisy_path = "data/Noisy"
save_model_path = "saved_models"
os.makedirs(save_model_path, exist_ok=True)

# Settings
batch_size = 8
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Read the labels CSV file
df = pd.read_csv(csv_file)
noise_label_map = {"Salt & Pepper": 0, "Gaussian": 1, "periodic": 2, "Periodic": 2}
df['label'] = df['noise_type'].map(noise_label_map)

# Data transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# Custom dataset class for noise classification
class NoiseClassificationDataset(Dataset):
    def __init__(self, df, noisy_path, transform=None):
        self.df = df
        self.noisy_path = noisy_path
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['image_name']
        label = self.df.iloc[idx]['label']
        noisy_img = Image.open(os.path.join(self.noisy_path, img_name)).convert("RGB")
        if self.transform:
            noisy_img = self.transform(noisy_img)
        return noisy_img, label

# Split dataset into train and test sets
train_size = int(0.8 * len(df))
test_size = len(df) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(
    NoiseClassificationDataset(df, noisy_path, transform),
    [train_size, test_size]
)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the ResNet-based model for noise classification
class NoiseClassifierResNet(nn.Module):
    def __init__(self):
        super(NoiseClassifierResNet, self).__init__()
        self.resnet = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.resnet.fc = nn.Linear(512, 3)  # 3 classes for noise types
    
    def forward(self, x):
        return self.resnet(x)

# Initialize model
model = NoiseClassifierResNet().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.6f}")

# Evaluation
model.eval()
predictions, ground_truths = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())
        ground_truths.extend(labels.numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(ground_truths, predictions)
precision = precision_score(ground_truths, predictions, average='macro')
recall = recall_score(ground_truths, predictions, average='macro')
f1 = f1_score(ground_truths, predictions, average='macro')

print(f" Accuracy: {accuracy:.4f}")
print(f" Precision: {precision:.4f}")
print(f" Recall: {recall:.4f}")
print(f" F1-Score: {f1:.4f}")

# Save the trained model
model_save_path = os.path.join(save_model_path, "resnet_noise_classifier.pth")
torch.save(model.state_dict(), model_save_path)
